In [86]:
import pandas as pd
from openai import OpenAI
from pinecone import Pinecone
from keys.keys import OPENAI_API_KEY, PINECONE_API_KEY, INDEX_NAME
from functions.query_index import query_index
from functions.read_docs import read_docs
from functions.generate_answer import generate_answer
from functions.query import query
# from functions.query import query

In [87]:
client = OpenAI(api_key=OPENAI_API_KEY)
model = 'text-embedding-ada-002'

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

In [88]:
file_name = "/Users/tashatanarugsachock/Desktop/shopperbot-llm/dataset/exam.xlsx" 
df = pd.DataFrame(data=pd.read_excel(io=file_name))

df.head(5)

,Type,Question,A,B,C,D,Answer Key
0,General Information,When is Siam Paragon opening hours?,Siam Paragon opens from 8:00 AM to 8:00 PM.,Siam Paragon opens from 9:00 AM to 9:00 PM.,Siam Paragon opens from 10:00 AM to 10:00 PM.,Siam Paragon opens from 11:00 AM to 11:00 PM.,C
1,General Information,Is there free Wi-Fi available in Siam Paragon?,"No, there is no Wi-Fi available in Siam Paragon.","Yes, but it's only available for staff members.","Yes, there is free Wi-Fi available in Siam Par...","Yes, but customers need to pay for Wi-Fi acces...",C
2,General Information,Are pets allowed in Siam Paragon?,"Yes, pets are welcome in Siam Paragon.","No, pets are not allowed in Siam Paragon.","Yes, but only small pets are allowed in Siam P...","Yes, but only if they are carried in pet carri...",B
3,General Information,What is the contact number for Siam Paragon's ...,02-610-8000,02-937-1000,02-937-1345,02-215-1000,A
4,General Information,Are there any prayer rooms available?,"No, there are no prayer rooms available in Sia...","Yes, there is a Muslim prayer room available o...","Yes, there is a prayer room available on the 3...","Yes, there is a prayer room available on the 2...",B


In [89]:
base = 'From this question, what is the correct choice - answer in this format X: the answer? Question: '

In [90]:
def dfquery(row, client, model, index):
    q = row['Question']
    a = row['A']
    b = row['B']
    c = row['C']
    d = row['D']
    prompt = base + q + ' A: ' + a + ' B: ' + b + ' C: ' + c + ' D: ' + d
    
    var_fn = {}
    
    index_result = query_index(client=client, model=model, index=index, prompt=prompt)
    var_fn.update({'index_result': index_result})
    
    file_ids = []
    for res in index_result:
        file_id = res.split(': ')[1]
        file_ids.append(file_id)
        
    contents = read_docs(file_ids=file_ids)
    
    response = generate_answer(client=client, prompt=prompt, contents=contents)
    var_fn.update({'response': response})
    
    answer = var_fn.get('response', '')
    answer = answer.replace('\r', '').replace('\n', '')
    answer = answer.replace('Answer: ', '')
   
    return answer

In [91]:
df['GPT'] = df.apply(dfquery, args=(client, model, index), axis = 1)

In [92]:
def ansformat(row):
    ans = row['GPT']
    parts = ans.split(': ')
    
    if len(ans) > 0:
        if len(parts) > 2 and (parts[1][-1] in ['A', 'B', 'C', 'D'] or 'correct' in parts[0]):
            return parts[1][-1]
        else:
            if parts[0][-1] in ['A', 'B', 'C', 'D']:
                return parts[0][-1]
            else:
                return 'X'
    
    
# A: B: answer
# The correct choice is: A: 
# .A:
# A: time is 10:00
# A: B: time is 10:00
# A: X: answer
    

In [93]:
df['Ansformatted'] = df.apply(ansformat, axis = 1)

In [96]:
def anscheck(row):
    
    if row['Answer Key'] == row['Ansformatted']:
        return 1
    else:
        return 0

In [97]:
df['Score'] = df.apply(anscheck, axis = 1)

In [99]:
df.head(50)

,Type,Question,A,B,C,D,Answer Key,GPT,Ansformatted,Score
0,General Information,When is Siam Paragon opening hours?,Siam Paragon opens from 8:00 AM to 8:00 PM.,Siam Paragon opens from 9:00 AM to 9:00 PM.,Siam Paragon opens from 10:00 AM to 10:00 PM.,Siam Paragon opens from 11:00 AM to 11:00 PM.,C,C: Siam Paragon opens from 10:00 AM to 10:00 PM.,C,1
1,General Information,Is there free Wi-Fi available in Siam Paragon?,"No, there is no Wi-Fi available in Siam Paragon.","Yes, but it's only available for staff members.","Yes, there is free Wi-Fi available in Siam Par...","Yes, but customers need to pay for Wi-Fi acces...",C,"C: Yes, there is free Wi-Fi available in Siam...",C,1
2,General Information,Are pets allowed in Siam Paragon?,"Yes, pets are welcome in Siam Paragon.","No, pets are not allowed in Siam Paragon.","Yes, but only small pets are allowed in Siam P...","Yes, but only if they are carried in pet carri...",B,"B: No, pets are not allowed in Siam Paragon.",B,1
3,General Information,What is the contact number for Siam Paragon's ...,02-610-8000,02-937-1000,02-937-1345,02-215-1000,A,C: 02-937-1345,C,0
4,General Information,Are there any prayer rooms available?,"No, there are no prayer rooms available in Sia...","Yes, there is a Muslim prayer room available o...","Yes, there is a prayer room available on the 3...","Yes, there is a prayer room available on the 2...",B,"A: A: No, there are no prayer rooms available ...",A,0
5,General Information,Is there a medical clinic or first aid station...,"No, there is no medical assistance available ...","Yes, there is a medical clinic located on the ...","Yes, First Aid is available for visitors in th...","Yes, there is a first aid station located on t...",C,"A: No, there is no medical assistance availabl...",A,0
6,General Information,Are there any lockers available for shoppers t...,"Yes, lockers are available near the main entra...","No, there are no lockers available for shopper...","Yes, lockers can be found on the G floor insid...","Yes, luggages can be dropped off at Lock Box P...",D,"D: Yes, luggages can be dropped off at Lock Bo...",D,1
7,General Information,Are there any designated smoking areas?,"No, smoking is not permitted anywhere in Siam ...","Yes, there is a smoking area on the ground floor.","Yes, there is a smoking area on the 2nd floor.","Yes, there is a smoking area on the 4th floor.",C,"A: No, smoking is not permitted anywhere in S...",A,0
8,General Information,What are the peak hours at Siam Paragon for sh...,9:00 AM to 12:00 PM,12:00 PM to 3:00 PM,3:00 PM to 6:00 PM,6:00 PM to 9:00 PM,C,D: 6:00 PM to 9:00 PM,D,0
9,Membership and Loyalty Program,How can I join the loyalty program?,By signing up at the customer service desk in ...,By downloading the Siam Paragon mobile app and...,By applying for credit card loyalty programs o...,By attending a membership enrollment event at ...,C,B: By downloading the Siam Paragon mobile app ...,B,0


In [100]:
df['Counter'] = 1

In [101]:
result_path = "/Users/tashatanarugsachock/Desktop/shopperbot-llm/dataset/result/result01.xlsx"

In [102]:
df.to_excel(result_path, index=False)